In [1]:
#load libraries

import gurobipy as gp
import numpy as np
import pandas as pd

In [2]:
# Define initial dataframe

ROIs = pd.read_csv('ROI_data.csv')
ROIs

,Platform,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [3]:
roi_mat = pd.read_csv('roi_mat.csv')

In [4]:
pd.read_csv('index_data.csv')

,Unnamed: 0,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,1,13,25,37,49,61,73,85,97,109
1,February,2,14,26,38,50,62,74,86,98,110
2,March,3,15,27,39,51,63,75,87,99,111
3,April,4,16,28,40,52,64,76,88,100,112
4,May,5,17,29,41,53,65,77,89,101,113
5,June,6,18,30,42,54,66,78,90,102,114
6,July,7,19,31,43,55,67,79,91,103,115
7,August,8,20,32,44,56,68,80,92,104,116
8,September,9,21,33,45,57,69,81,93,105,117
9,October,10,22,34,46,58,70,82,94,106,118


# 3. Formulate the marketing budget allocation problem as a linear program. Use gurobi to find the optimal budget allocation.

In [5]:
# Define constants

budget = 10

In [6]:
# Find optimal solution for initial consulting firm's estimates

model1 = gp.Model()
model1X = model1.addMVar(10, ub = 3)

model1.setObjective(gp.quicksum(ROIs.iloc[0][i+1]*model1X[i] for i in range(10)), sense=gp.GRB.MAXIMIZE)

model1.addConstr((model1X[0] + model1X[1]) <= (model1X[4] + model1X[9]))
model1.addConstr(gp.quicksum(model1X[i] for i in range(4, 9)) >= 2 * (model1X[2] + model1X[3]))
model1.addConstr(gp.quicksum(model1X[i] for i in range(10)) <= 10)


Restricted license - for non-production use only - expires 2024-10-28


<MConstr () *awaiting model update*>

In [7]:
model1.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 10 columns and 21 nonzeros
Model fingerprint: 0xcac47b03
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [3e+00, 3e+00]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.00s
Presolved: 3 rows, 10 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e-01   1.700000e+01   0.000000e+00      0s
       1    4.5600000e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.560000000e-01


In [8]:
model1.x

[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]

In [9]:
model1.objVal

0.45600000000000007

In [10]:
ROIs

,Platform,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [11]:
model2 = gp.Model()
model2X = model2.addMVar(10, ub = 3)

model2.setObjective(gp.quicksum(ROIs.iloc[1][i+1]*model2X[i] for i in range(10)), sense=gp.GRB.MAXIMIZE)

conlist = [0]*3
conlist[0] = model2.addConstr((model2X[0] + model2X[1]) <= (model2X[4] + model2X[9]))
conlist[1] = model2.addConstr(gp.quicksum(model2X[i] for i in range(4, 9)) >= 2 * (model2X[2] + model2X[3]))
conlist[2] = model2.addConstr(gp.quicksum(model2X[i] for i in range(10)) <= 10)


In [12]:
model2.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 10 columns and 21 nonzeros
Model fingerprint: 0x2bfada38
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-02, 5e-02]
  Bounds range     [3e+00, 3e+00]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.00s
Presolved: 3 rows, 10 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e-01   1.700000e+01   0.000000e+00      0s
       1    4.5600000e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.560000000e-01


In [13]:
model2.x

[3.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0]

In [14]:
model2.objval

0.45600000000000007

## 5a. Are the allocations the same?

In [15]:
model1.x

[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]

In [16]:
model2.x

[3.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0]

In [17]:
model1.objVal

0.45600000000000007

In [18]:
model2.objval

0.45600000000000007

The allocations are not the same, but the objective value is.

## 5b Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)? 

In [19]:
sum(model2.x * ROIs.iloc[0][1:])

0.252

In [20]:
# Difference

sum(model2.x * ROIs.iloc[0][1:]) - model1.ObjVal


-0.20400000000000007

By using the second allocation for the first returns, the objective value is 0.204 less than what it would have been under the first allocation.

## 5c. Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective? 

In [21]:
sum(model1.x*ROIs.iloc[1][1:])

0.264

In [22]:
sum(model1.x*ROIs.iloc[1][1:]) - model2.objval

-0.19200000000000006

The first allocation is .192 lower than the second allocation if the first assumptions are correct.

The third constraint is not useful is binding and decreases the maximum possible return. 

In [23]:
model1.x

[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]

In [24]:
model1.SAObjLow

[-inf, 0.039, -inf, 0.033, -inf, -inf, 0.039, -inf, -inf, 0.028999999999999998]

In [25]:
model1.SAObjUp

[0.049,
 0.062,
 0.039,
 0.046,
 0.028999999999999998,
 0.039,
 inf,
 0.039,
 0.039,
 inf]

## 7. Your boss has gained permission to reinvest half of the return. For example, if the marketing obtains a 4% return in January, the budget of February will be $10M + $10M × 4% × 50% = $10.2M. The monthly ROI for next year is given in an attached csv file. The three constraints given by your boss are still in place for each month. What is the optimal allocation for each month?

In [26]:
budget = 10
output = pd.DataFrame()

for month in range(12):

    model = gp.Model()
    modelX = model.addMVar(10, ub = 3)

    model.setObjective(gp.quicksum((roi_mat.iloc[month, i+1])/100*modelX[i] for i in range(10)), sense=gp.GRB.MAXIMIZE)

    conlist = [0]*3
    conlist[0] = model.addConstr((modelX[0] + modelX[1]) <= (modelX[4] + modelX[9]))
    conlist[1] = model.addConstr(gp.quicksum(modelX[i] for i in range(4, 9)) >= 2 * (modelX[2] + modelX[3]))
    conlist[2] = model.addConstr(gp.quicksum(modelX[i] for i in range(10)) <= budget)
    
    model.Params.OutputFlag = 0 
    model.optimize()
    print(model.x)
    print(model.objval)
    
    values = model.x
    output = output.append(pd.Series(values), ignore_index=True)

    budget += (model.objval/2)


[3.0, 0.0, 0.0, 1.3333333333333333, 0.0, 0.0, 2.666666666666667, 0.0, 0.0, 3.0]
0.373
[3.0, 0.0, 0.0, 2.3955, 3.0, 0.0, 0.0, 0.0, 1.7910000000000004, 0.0]
0.40629600000000005
[0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 1.389648000000001, 0.0, 3.0, 0.0]
0.41441697600000005
[0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 1.5968564880000002, 0.0]
0.414486833568
[1.8040999047839996, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 3.0]
0.43214349666744
[3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0201716531177194, 3.0]
0.45476652281847335
[1.1237774572634782, 0.0, 0.0, 3.0, 1.1237774572634782, 0.0, 3.0, 0.0, 3.0, 0.0]
0.4686546416665513
[3.0, 0.0, 0.0, 1.8272940784534104, 0.0, 0.654588156906821, 0.0, 0.0, 3.0, 3.0]
0.4879661131005957
[1.362932645955265, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 3.0, 1.362932645955265]
0.4592199484466449
[0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 3.0, 0.0, 0.0, 2.9554752661338526]
0.4275752085162833
[3.0, 0.0, 0.0, 2.0564209567973313, 0.0, 1.1128419135946632, 3.0, 0.0, 0.0, 3.0]
0.5173755681180985
[3.0, 3.

/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/1022525256.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/1022525256.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/1022525256.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/1022525256.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [27]:
output

,0,1,2,3,4,5,6,7,8,9
0,3.000000,0.0,0.0,1.333333,0.000000,0.000000,2.666667,0.0,0.000000,3.000000
1,3.000000,0.0,0.0,2.395500,3.000000,0.000000,0.000000,0.0,1.791000,0.000000
2,0.000000,0.0,0.0,3.000000,0.000000,3.000000,1.389648,0.0,3.000000,0.000000
3,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,1.596856,0.000000
4,1.804100,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,3.000000,3.000000
5,3.000000,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,2.020172,3.000000
6,1.123777,0.0,0.0,3.000000,1.123777,0.000000,3.000000,0.0,3.000000,0.000000
7,3.000000,0.0,0.0,1.827294,0.000000,0.654588,0.000000,0.0,3.000000,3.000000
8,1.362933,0.0,0.0,3.000000,0.000000,3.000000,0.000000,0.0,3.000000,1.362933
9,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,0.000000,2.955475


## 8. A stable budget is defined as a monthly allocation such that for each platform the monthly change in spend is no more than $1M. Is the allocation you found stable? If it isn’t, you do not need to solve a new optimization model. Describe how my might model this?

In the for loop, you could create a variable that stores the previous allocations and create new contraints that ensure the new variables are within $1M of the previous allocation.

In [28]:
budget = 10
output = pd.DataFrame()

for month in range(12):

    model = gp.Model()
    modelX = model.addMVar(10, ub = 3)

    model.setObjective(gp.quicksum((roi_mat.iloc[month, i+1])/100*modelX[i] for i in range(10)), sense=gp.GRB.MAXIMIZE)

    conlist = [0]*5
    conlist[0] = model.addConstr((modelX[0] + modelX[1]) <= (modelX[4] + modelX[9]))
    conlist[1] = model.addConstr(gp.quicksum(modelX[i] for i in range(4, 9)) >= 2 * (modelX[2] + modelX[3]))
    conlist[2] = model.addConstr(gp.quicksum(modelX[i] for i in range(10)) <= budget)

    model.Params.OutputFlag = 0 
    if month == 0:
        model.optimize()
    else:
        conlist[3] = model.addConstrs(modelX[i] - values[i] <= 1 for i in range(10))
        conlist[4] = model.addConstrs(modelX[i] - values[i] >= -1 for i in range(10))
        model.optimize()

    print(model.x)
    print(model.objval)
    
    values = model.x
    output = output.append(pd.Series(values), ignore_index=True)

    budget += (model.objval/2)



[3.0, 0.0, 0.0, 1.3333333333333333, 0.0, 0.0, 2.666666666666667, 0.0, 0.0, 3.0]
0.373
[3.0, 0.0, 0.0, 1.7288333333333334, 1.0, 0.0, 1.666666666666667, 0.0, 0.7910000000000001, 2.0]
0.37296266666666666
[2.0, 0.0, 0.0, 2.124327111111111, 0.0, 1.0, 1.4576542222222222, 0.0, 1.7910000000000001, 2.0]
0.3708796364444444
[1.0, 0.0, 0.0, 2.852807050518519, 0.0, 2.0, 2.457654222222222, 0.0, 1.247959878814815, 1.0]
0.40438327208237046
[0.6598458582634072, 0.0, 0.0, 1.852807050518519, 0.0, 1.0, 3.0, 0.0, 2.247959878814815, 2.0]
0.40924823930351556
[1.6598458582634072, 0.0, 0.0, 0.852807050518519, 0.0, 0.0, 3.0, 0.0, 2.4525839984665727, 3.0]
0.44658746598989574
[1.3357235897249287, 0.0, 0.0, 1.852807050518519, 0.0, 0.0, 3.0, 0.0, 3.0, 2.0]
0.4586167302220871
[0.9178390053544909, 0.0, 0.0, 2.5, 0.0, 0.0, 2.0, 0.0, 3.0, 3.0]
0.47654923822488865
[1.917839005354491, 0.0, 0.0, 2.5, 0.0, 1.0, 1.0, 0.0, 3.0, 2.2382746191124436]
0.4447327362693572


/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pa

[0.9178390053544909, 0.0, 0.0, 3.0, 0.0, 2.0, 2.0, 0.0, 2.0, 1.9606409872471229]
0.4082756817525427
[1.917839005354491, 0.0, 0.0, 2.5, 0.0, 1.0, 3.0, 0.0, 1.0, 2.664778828123394]
0.49681220421007477
[2.917839005354491, 1.0, 0.0, 1.8043949767428102, 1.0, 0.0, 2.0, 0.0, 0.6087899534856203, 3.0]
0.4737353247219819


/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_40754/3319818658.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.Series(values), ignore_index=True)


In [29]:
output

,0,1,2,3,4,5,6,7,8,9
0,3.000000,0.0,0.0,1.333333,0.0,0.0,2.666667,0.0,0.000000,3.000000
1,3.000000,0.0,0.0,1.728833,1.0,0.0,1.666667,0.0,0.791000,2.000000
2,2.000000,0.0,0.0,2.124327,0.0,1.0,1.457654,0.0,1.791000,2.000000
3,1.000000,0.0,0.0,2.852807,0.0,2.0,2.457654,0.0,1.247960,1.000000
4,0.659846,0.0,0.0,1.852807,0.0,1.0,3.000000,0.0,2.247960,2.000000
5,1.659846,0.0,0.0,0.852807,0.0,0.0,3.000000,0.0,2.452584,3.000000
6,1.335724,0.0,0.0,1.852807,0.0,0.0,3.000000,0.0,3.000000,2.000000
7,0.917839,0.0,0.0,2.500000,0.0,0.0,2.000000,0.0,3.000000,3.000000
8,1.917839,0.0,0.0,2.500000,0.0,1.0,1.000000,0.0,3.000000,2.238275
9,0.917839,0.0,0.0,3.000000,0.0,2.0,2.000000,0.0,2.000000,1.960641
